In [1]:
import pymanopt
import numpy as np

dim=3

In [2]:
man = pymanopt.manifolds.Sphere(dim)
A = np.random.rand(dim-1, dim-1)
cov = np.dot(A, A.transpose())
mean = man.random_point()

In [3]:
mean, cov

(array([-0.44328158, -0.46141816,  0.76850161]),
 array([[1.26839351, 0.32717228],
        [0.32717228, 0.13089927]]))

In [272]:
# generate tangent space
N = 100
samples = np.random.multivariate_normal(np.zeros(dim-1), cov, N)

In [273]:
# We need a tangent
normal = mean/np.linalg.norm(mean)

In [274]:
tol = 1e-5

def gram_schmidt(vectors):
    Q = np.zeros_like(vectors)
    for i in range(len(vectors)):
        v = vectors[i]
        for j in range(i):
            proj = np.dot(v, Q[j]) / np.dot(Q[j], Q[j]) * Q[j]
            v = v - proj
        Q[i] = v / np.linalg.norm(v)
    return Q
    
def get_axis(normal):
    # Assumes it is norm 1
    Q = np.eye(dim)
    arg = np.argmax(normal)
    if normal[arg]>(1-tol) and np.sum(normal)>(1-tol):
        return np.concatenate([Q[:arg], Q[arg+1:]])
    return gram_schmidt(np.vstack([normal,Q[:-1]]))[1:]

def get_e(i,n):
    return np.array(i*[0]+[1]+(n-i-1)*[0])

In [281]:
ax = get_axis(normal)

# Project points onto the axis to get all points in the tangent space
tangent_points = samples@ax

points = man.exp(mean, tangent_points)

In [282]:
# For estimateing normalisation constant
def estimate_normalising(mu, Sigma, manifold, m=None, S=100):
    d = manifold.dim
    z = np.sqrt((2*np.pi)**d*np.linalg.det(Sigma))
    samples = np.random.multivariate_normal(np.zeros(d), Sigma, S)
    ax = get_axis(mu)
    vs = samples@ax
    return  z*np.mean(compute_vol(mu, vs, manifold, m))

sphere_M = lambda v: np.array([np.eye(v.shape[-1])]*len(v))

def compute_vol(mu, v, manifold, m=None):
    if m == None: m = sphere_M
    return np.sqrt(np.linalg.det(m(manifold.exp(mu, v-mu))))

In [283]:
estimate_normalising(mean, cov, man)

np.float64(1.5260520812767422)

In [284]:
def extrinsic_to_log(manifold, mu, x, ax):
    point = manifold.log(mu, x)
    return np.dot(point, ax.T)

In [289]:
def objective_grad(points, mu, Sigma, manifold, S=100):
    d = manifold.dim
    samples = np.random.multivariate_normal(np.zeros(d), Sigma, S)
    ax = get_axis(mu)
    vs = samples@ax
    ms = compute_vol(mu, vs, manifold)
    z = np.sqrt((2*np.pi)**d*np.linalg.det(Sigma))
    grad = -(np.array([extrinsic_to_log(manifold,mu,p,ax) for p in points])
             .mean(0)-z*(ms.reshape(1,-1)@vs)/S)
    return grad
    
def mle_manifold(points, manifold):
    pass

In [286]:
ss = np.random.multivariate_normal(np.zeros(dim-1), cov, 100)
vs = ss@ax
ms = compute_vol(mean, vs, man)
z = np.sqrt((2*np.pi)**2*np.linalg.det(cov))
grad = -(np.array([extrinsic_to_log(man,mean,p,ax) for p in points]).mean(0)
         -z*(ms.reshape(1,-1)@ss)/(100*estimate_normalising(mean,cov,man)))
np.linalg.inv(cov)@grad.reshape(-1,1)

array([[ 0.20447833],
       [-1.19099027]])

In [287]:
grad

array([[-0.13030002, -0.08900011]])

In [269]:
vs[0]

array([1.65955897, 0.45228425, 1.22881211])

In [137]:
objective_grad(points, mean, cov, man)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 3 is different from 2)